In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
import numpy as np

In [3]:
import os
os.chdir("../")

In [4]:
import copy

In [5]:
words = open('data/names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [6]:
len(words)

32033

In [7]:
## building vocab of chars & mapping to/from integers

chars = sorted(list(set(''.join(words))))
stoi = {s:i + 1 for i,s in enumerate(chars)}
stoi["."] = 0
itos = {i:s for s,i in stoi.items()}

In [8]:
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [9]:
## Building the data set

block_size = 3 ## context length: How many chars to take to predict the next one?

X, Y = [], []

for w in words[0:5]:
    print(w)
    context = [0] * block_size # -> 1st Iter [0, 0, 0]
    
    for ch in w + ".":

        ix = stoi[ch] ## Encoded values of character
        
        X.append(context) ## 1st Iter [0, 0, 0]
        Y.append(ix) ## 1st Iter [itos[ch]]

        print(''.join(itos[i] for i in context), '---->', itos[ix])
        context = context[1:] + [ix] ## Rolling Operation

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ----> e
..e ----> m
.em ----> m
emm ----> a
mma ----> .
olivia
... ----> o
..o ----> l
.ol ----> i
oli ----> v
liv ----> i
ivi ----> a
via ----> .
ava
... ----> a
..a ----> v
.av ----> a
ava ----> .
isabella
... ----> i
..i ----> s
.is ----> a
isa ----> b
sab ----> e
abe ----> l
bel ----> l
ell ----> a
lla ----> .
sophia
... ----> s
..s ----> o
.so ----> p
sop ----> h
oph ----> i
phi ----> a
hia ----> .


#### Understanding Generated X Shape & Y Shape

1. X shape is (N, Context Size). Where N is number of input samples generated while creating X, Y Pairs and Context Size is defined above

2. Y Shape is (N). For each Input Sample, it contains the next character Output

In [10]:
X.shape, X.dtype, 

(torch.Size([32, 3]), torch.int64)

In [11]:
Y.shape, Y.dtype

(torch.Size([32]), torch.int64)

## Projecting characters into 2 Dimensional Space (Embedding Look Up Table)

Number of Chars: 27

Dimensions: 2

In [12]:
C = torch.randn((27, 2))

### Accessing entries of Look-Up Matrix C

In [13]:
## For 5 encoded char "e"
## itos[5]
C[5]

tensor([ 1.1134, -0.3061])

In [14]:
## For 0 encoded char "a"
## itos[5]
C[0]

tensor([ 0.5072, -0.1040])

In [15]:
## Is there a better way to index Look Up Matrix?
## Yes One Hot Encoding
F.one_hot(torch.tensor(5), num_classes=27).float() @ C

tensor([ 1.1134, -0.3061])

In [16]:
## First Layer of Neural Language Model is essentially, look up into Embedding matrices for Provide Input

### Getting values for 5th Row & 0th Row simultaneously
C[[5, 0]]

tensor([[ 1.1134, -0.3061],
        [ 0.5072, -0.1040]])

In [17]:
### Getting values for 5th Row , 6th Row & 7th Row ,7th Row, 7th Row simultaneously

C[torch.tensor([5, 6, 7, 7, 7])]

tensor([[ 1.1134, -0.3061],
        [-1.3514, -1.9442],
        [-0.5752, -0.0981],
        [-0.5752, -0.0981],
        [-0.5752, -0.0981]])

In [18]:
X.shape, X.dtype, 

(torch.Size([32, 3]), torch.int64)

### How can I get Embedding Look Up entries for X Input `[N, Context Size]`

In [19]:
## Slicing X Tensor for Single word emma (1st name in data)
X_single_name = X[0:4, :]

In [20]:
X_single_name

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13]])

In [21]:
## In each row of X
## Number of encoded chars == context sizr
## For each encodeed chars -> (, 2) shape Look-Up Embedding Table output

## End Shape would look like
## (N, Context_Size, Embedding Dim)
C[X_single_name], C[X_single_name].shape

(tensor([[[ 0.5072, -0.1040],
          [ 0.5072, -0.1040],
          [ 0.5072, -0.1040]],
 
         [[ 0.5072, -0.1040],
          [ 0.5072, -0.1040],
          [ 1.1134, -0.3061]],
 
         [[ 0.5072, -0.1040],
          [ 1.1134, -0.3061],
          [ 0.3804, -1.0019]],
 
         [[ 1.1134, -0.3061],
          [ 0.3804, -1.0019],
          [ 0.3804, -1.0019]]]),
 torch.Size([4, 3, 2]))

In [22]:
X[13, 2]

tensor(1)

In [23]:
C[X][13,2]

tensor([ 1.5127, -0.1777])

In [24]:
C[1]

tensor([ 1.5127, -0.1777])

In [25]:
## We can embedd directly by passing X into C and it should work!

emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [26]:
### Constructing Hidden Layer

W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [27]:
### Embedding which is input to Hidden Layer has shape of [N, context, embedding dimension] -> (32, 3, 2)
### While Hidden layer neuron shape is (context * embedding dimension, hidden_dim) -> (6, 100)

### Last 2 dimensions of embedding layer needs to combined together so that shape matches for matmul

### One way to solve is to use torch cat function

torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], dim=1).shape

torch.Size([32, 6])

In [28]:
torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], dim=1).shape

torch.Size([32, 6])

In [29]:
### More generic way would be use torch unbind and then use torch cat
### Unbinding is done on dimension where context size is defined

torch.cat(torch.unbind(emb, dim=1), dim=1)

tensor([[ 0.5072, -0.1040,  0.5072, -0.1040,  0.5072, -0.1040],
        [ 0.5072, -0.1040,  0.5072, -0.1040,  1.1134, -0.3061],
        [ 0.5072, -0.1040,  1.1134, -0.3061,  0.3804, -1.0019],
        [ 1.1134, -0.3061,  0.3804, -1.0019,  0.3804, -1.0019],
        [ 0.3804, -1.0019,  0.3804, -1.0019,  1.5127, -0.1777],
        [ 0.5072, -0.1040,  0.5072, -0.1040,  0.5072, -0.1040],
        [ 0.5072, -0.1040,  0.5072, -0.1040, -0.8352, -0.8074],
        [ 0.5072, -0.1040, -0.8352, -0.8074,  0.9148,  1.2704],
        [-0.8352, -0.8074,  0.9148,  1.2704, -0.0758,  0.6936],
        [ 0.9148,  1.2704, -0.0758,  0.6936,  0.6273, -1.5315],
        [-0.0758,  0.6936,  0.6273, -1.5315, -0.0758,  0.6936],
        [ 0.6273, -1.5315, -0.0758,  0.6936,  1.5127, -0.1777],
        [ 0.5072, -0.1040,  0.5072, -0.1040,  0.5072, -0.1040],
        [ 0.5072, -0.1040,  0.5072, -0.1040,  1.5127, -0.1777],
        [ 0.5072, -0.1040,  1.5127, -0.1777,  0.6273, -1.5315],
        [ 1.5127, -0.1777,  0.6273, -1.5

In [30]:
Input_Embedding_Proj = copy.deepcopy(C[X])

In [31]:
Input_Embedding_Proj.shape

torch.Size([32, 3, 2])

In [32]:
#### Easier way to do this

a = torch.arange(18)
a 

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17])

In [33]:
a.view(2, 9)

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13, 14, 15, 16, 17]])

In [34]:
a.view(9, 2)

tensor([[ 0,  1],
        [ 2,  3],
        [ 4,  5],
        [ 6,  7],
        [ 8,  9],
        [10, 11],
        [12, 13],
        [14, 15],
        [16, 17]])

In [41]:
emb.shape

torch.Size([32, 3, 2])

In [37]:
emb.view(32,6)

tensor([[ 0.5072, -0.1040,  0.5072, -0.1040,  0.5072, -0.1040],
        [ 0.5072, -0.1040,  0.5072, -0.1040,  1.1134, -0.3061],
        [ 0.5072, -0.1040,  1.1134, -0.3061,  0.3804, -1.0019],
        [ 1.1134, -0.3061,  0.3804, -1.0019,  0.3804, -1.0019],
        [ 0.3804, -1.0019,  0.3804, -1.0019,  1.5127, -0.1777],
        [ 0.5072, -0.1040,  0.5072, -0.1040,  0.5072, -0.1040],
        [ 0.5072, -0.1040,  0.5072, -0.1040, -0.8352, -0.8074],
        [ 0.5072, -0.1040, -0.8352, -0.8074,  0.9148,  1.2704],
        [-0.8352, -0.8074,  0.9148,  1.2704, -0.0758,  0.6936],
        [ 0.9148,  1.2704, -0.0758,  0.6936,  0.6273, -1.5315],
        [-0.0758,  0.6936,  0.6273, -1.5315, -0.0758,  0.6936],
        [ 0.6273, -1.5315, -0.0758,  0.6936,  1.5127, -0.1777],
        [ 0.5072, -0.1040,  0.5072, -0.1040,  0.5072, -0.1040],
        [ 0.5072, -0.1040,  0.5072, -0.1040,  1.5127, -0.1777],
        [ 0.5072, -0.1040,  1.5127, -0.1777,  0.6273, -1.5315],
        [ 1.5127, -0.1777,  0.6273, -1.5

In [ ]:
h = torch.tanh(emb.view(emb.shape[0], 6) @ W1 + b1)

In [48]:
h

tensor([[ 0.5875, -0.2874,  0.7035,  ..., -0.9074, -0.0702, -0.5372],
        [ 0.9355, -0.2456,  0.7806,  ..., -0.6357,  0.4462, -0.9251],
        [ 0.7620,  0.1015,  0.5099,  ..., -0.8211,  0.5410, -0.7211],
        ...,
        [-0.9969, -0.9986,  0.2435,  ..., -1.0000,  0.9730, -1.0000],
        [ 0.9076,  0.9745, -0.3043,  ..., -0.9998, -0.9988,  1.0000],
        [ 0.9404, -0.5978,  0.9007,  ...,  0.9568,  0.0358, -0.9983]])

In [49]:
### Output Layer

W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [50]:
logits = h @ W2 + b2

In [60]:
loss = F.cross_entropy(logits, Y)

In [61]:
loss

tensor(15.9168)

#### Training Loop 

In [63]:
X.shape, Y.shape ## DataSet

(torch.Size([32, 3]), torch.Size([32]))

In [70]:
g = torch.Generator().manual_seed(2147483647)

C = torch.randn((27, 2), generator=g, requires_grad=True)

W1 = torch.randn((6, 100), generator=g, requires_grad=True)
b1 = torch.randn(100, generator=g, requires_grad=True)

W2 = torch.randn((100, 27), generator=g, requires_grad=True)
b2 = torch.randn(27, generator=g, requires_grad=True)

parameters = [C, W1, b1, W2, b2]

In [71]:
sum(p.nelement() for p in parameters)

3481

In [72]:
for _ in range(10):


    # Forward Pass
    emb = C[X]

    h = torch.tanh(emb.view(emb.shape[0], 6) @ W1 + b1)

    logits = h @ W2 + b2

    loss = F.cross_entropy(logits, Y)
    print(loss.item())

    ## Backward pass 

    for p in parameters:
        p.grad = None

    loss.backward()

    ## Update
    for p in parameters:
        p.data += -0.1 * p.grad



17.76971435546875
13.656402587890625
11.298770904541016
9.4524564743042
7.984263896942139
6.891321659088135
6.1000142097473145
5.452035903930664
4.898152828216553
4.414663791656494


In [68]:
loss

tensor(17.7697)